# Lecture 14, Algebraic Modeling Languages

Algebraic Modeling Languages (AML) are high-level computer programming languages for describing and solving high complexity problems for large scale mathematical computation (i.e. large scale optimization type problems).  Their syntax mimics the mathematical notation of optimization problems, which allows one to express optimization problems in a familiar, concise and readable way. 

**AMLs do not directly solve the problem, but they call appropriate external solvers to find the solution.**

Examples of AMLs are
* A Mathematical Programming Language (AMPL),
* General Algebraic Modeling System (GAMS),
* Optimization Programming Language (OPL),
* Advanced Interactive Multidimensional Modeling System (AIMMS), and
* Pyomo.

In addition to the ease of modelling, one of the advantages of AMLs is that you can model the problem once and then solve it with multiple solvers.

## Pyomo

On this course, we use Pyomo as an example of AMLs. Pyomo is a Python-based, open-source optimization modeling language with a diverse set of optimization capabilities.

Pyomo may not be a completely typical AML, because Pyomo's modeling objects are embedded within a full-featured high-level programming language providing a rich set of supporting libraries, which distinguishes Pyomo from other AMLs.

Pyomo supports a wide range of problem types, including:
* Linear programming
* Quadratic programming
* Nonlinear programming
* Mixed-integer linear programming
* Mixed-integer quadratic programming
* Mixed-integer nonlinear programming
* Stochastic programming
* Generalized disjunctive programming
* Differential algebraic equations
* Bilevel programming
* Mathematical programs with equilibrium constraints

# Installing Pyomo

The easiest way to install Pyomo is to call
```
pip install pyomo
```
when pip has been installed on your machine.

## Example 1, linear optimization

Let us start with a very simple linear problem
$$
\begin{align}
\min &\qquad   2x_1+3x_2\\
\text{s.t. }& \qquad 2x_1+3x_2\geq 1\\
& \qquad x_1,x_2\geq 0.
\end{align}
$$

In [12]:
from pyomo.environ import *


model = ConcreteModel()

model.x = Var([1,2], domain=NonNegativeReals) #Non-negative variables x[1] and x[2]

model.OBJ = Objective(expr = 2*model.x[1] + 3*model.x[2]) #Objective function

model.Constraint1 = Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1) #Constraint


Once we have defined the problem, we can solve it. Let us start by using glpk, which is an open source linear programming program.

You need to have glpk installed on your system. For details, see https://www.gnu.org/software/glpk/#TOCdownloading. For many Linux distributions, you can install glpk from the repositories by typing
```
sudo yum install glpk
```
```
sudo apt-get install glpk,
```
or whatever your distribution needs.


In [5]:
from pyomo.opt import SolverFactory #Import interfaces to solvers
opt = SolverFactory("glpk") #Use glpk
res = opt.solve(model, tee=True) #Solve the  problem and print the output
print "Solution:"
print "========="
model.x.display() #Print values of x

GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 --write /tmp/tmpAfP0nw.glpk.raw --wglp /tmp/tmpOz7bG8.glpk.glp --cpxlp /tmp/tmpnK7MYT.pyomo.lp
Reading problem data from '/tmp/tmpnK7MYT.pyomo.lp'...
2 rows, 3 columns, 3 non-zeros
21 lines were read
Writing problem data to `/tmp/tmpOz7bG8.glpk.glp'...
15 lines were written
GLPK Simplex Optimizer, v4.52
2 rows, 3 columns, 3 non-zeros
Preprocessing...
1 row, 2 columns, 2 non-zeros
Scaling...
 A: min|aij| =  3.000e+00  max|aij| =  4.000e+00  ratio =  1.333e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
      0: obj =   0.000000000e+00  infeas =  1.000e+00 (0)
*     1: obj =   7.500000000e-01  infeas =  0.000e+00 (0)
*     2: obj =   6.666666667e-01  infeas =  0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40408 bytes)
Writing basic solution to `/tmp/tmpAfP0nw.glpk.raw'...
7 lines were written
Solution:
x : Size=2, Index=x_i

Now, if you have other linear solvers installed on your system, you can use them too. Let us use Cplex, which is a commercial solvers (academic license available).

In [6]:
opt = SolverFactory("cplex")
res = opt.solve(model, tee=True)
print "Solution:"
model.x.display()


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.6.2.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2015.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpaGJg1Q.cplex.log' open.
CPLEX> Problem '/tmp/tmpYBzsUS.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Problem name         : /tmp/tmpYBzsUS.pyomo.lp
Objective sense      : Minimize
Variables            :       3
Objective nonzeros   :       2
Linear constraints   :       2  [Greater: 1,  Equal: 1]
  Nonzeros           :       3
  RHS nonzeros       :       2

Variables            : Min LB: 0.000000         Max UB: all infinite   
Objective nonzeros   : Min   : 2.000000         Max   : 3.000000       
Linear constraints   :
  Nonzeros           : Min   : 1.000000         Max 

We can use also gurobi, which is another commercial solver with academic license.

In [13]:
opt = SolverFactory("gurobi")
res = opt.solve(model, tee=True)
print "Solution:"
model.x.display()

Optimize a model with 2 rows, 3 columns and 3 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 4e+00]
  Objective range [2e+00, 3e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 1e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6666667e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  6.666666667e-01
Solution:
x : Size=2, Index=x_index, Domain=NonNegativeReals
    Key : Lower : Value          : Upper : Fixed : Stale
      1 :     0 : 0.333333333333 :  None : False : False
      2 :     0 :            0.0 :  None : False : False


## Example 2, nonlinear optimization

Let use define optimizaiton problem
$$
\begin{align}
\min &\qquad c_d\\
\text{s.t. }& \qquad c_{af}*s_v - s_v*c_a-k_1*c_a=0\\
&\qquad s_v*c_b+k_1*c_a-k_2*c_b=0\\
&\qquad s_v*c_c+k_2*c_b=0\\
&\qquad s_v*c_d+k_3*c_a^2=0,\\
&\qquad s_v,c_a,c_b,c_c,c_d\geq0
\end{align}
$$
where $k_1=5/6$, $k_2=5/3$, $k_3=1/6000$, and $c_{af}=10000$.

In [8]:
from pyomo.environ import *
# create the concrete model
model = ConcreteModel()
# set the data (native python data)
k1 = 5.0/6.0 # min-1
k2 = 5.0/3.0 # min-1
k3 = 1.0/6000.0 # m3/(gmol min)
caf = 10000.0 # gmol/m3
# create the variables
model.sv = Var(initialize = 1.0, within=PositiveReals)
model.ca = Var(initialize = 5000.0, within=PositiveReals)
model.cb = Var(initialize = 2000.0, within=PositiveReals)
model.cc = Var(initialize = 2000.0, within=PositiveReals)
model.cd = Var(initialize = 1000.0, within=PositiveReals)

# create the objective
model.obj = Objective(expr = model.cb, sense=maximize)
# create the constraints
model.ca_bal = Constraint(expr = (0 == model.sv * caf \
    - model.sv * model.ca - k1 * model.ca \
    - 2.0 * k3 * model.ca ** 2.0))
model.cb_bal = Constraint(expr=(0 == -model.sv * model.cb \
    + k1 * model.ca - k2 * model.cb))
model.cc_bal = Constraint(expr=(0 == -model.sv * model.cc \
    + k2 * model.cb))
model.cd_bal = Constraint(expr=(0 == -model.sv * model.cd \
    + k3 * model.ca ** 2.0))

## Solving with Ipopt

Install IPopt following http://www.coin-or.org/Ipopt/documentation/node10.html.

In [11]:
opt = SolverFactory("ipopt",solver_io="nl")

opt.solve(model,tee=True)

print "Solution is "
model.sv.display()
model.ca.display()
model.cb.display()
model.cc.display()
model.cd.display()



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       11
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        5
                     variables with only lower bounds:        5
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

## Black-box optimization problem

In [ ]:
import sys
from pyomo.opt.blackbox import RealOptProblem

class RealProblem1(RealOptProblem):

    def __init__(self):
        RealOptProblem.__init__(self)
        self.lower=[0.0, -1.0, 1.0, None]
        self.upper=[None, 0.0, 2.0, -1.0]
        self.nvars=4

    def function_value(self, point):
        self.validate(point)
        return point.vars[0] - point.vars[1] + (point.vars[2]-1.5)**2 + (point.vars[3]+2)**4

problem = RealProblem1()
